In [16]:
pip install -r ../services/ml_training/requirements.txt

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
     ---------------------------------------- 0.0/642.7 kB ? eta -:--:--
     ---- ---------------------------------- 81.9/642.7 kB 2.3 MB/s eta 0:00:01
     ------------- ------------------------ 235.5/642.7 kB 3.6 MB/s eta 0:00:01
     ----------------------- -------------- 399.4/642.7 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 642.7/642.7 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
     ---------------------------------------- 0.0/42.9 kB ? eta

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Celda 1: Instalación de Dependencias (Con el comando mágico %)
# Forzamos una versión de matplotlib estable y D-Tale
%pip install pandas numpy seaborn matplotlib==3.7.3 sqlalchemy psycopg2-binary dtale

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.5 MB 2.6 MB/s eta 0:00:03
   - -------------------------------------- 0.3/7.5 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/7.5 MB 5.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 7.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.0/7.5 MB 5.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.5 MB/s eta 0:00:02
   -------- ------------------------------- 1.6/7.5 MB 5.5 MB/s eta 0:00:02
   ----------- -------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# **🟢 PARTE 1: ANÁLISIS MACRO**
Objetivo: Entender el terreno de juego global antes de mirar a los jugadores. Validar la calidad del dato y definir el contexto. Para ello se realizarán los siguiente cálculos:

## 1. Auditoría de Integridad (Data Health)
    - Mapa de Calor de Nulos
    - Cardinalidad de Variables
    - Detección de Duplicados Ocultos
## 2. La "Frontera de la Relevancia" (Análisis de Volumen)
    - Histograma de Minutos Jugados + Curva de Densidad
    - Distribución de Edad
    - Pareto del Juego
## 3. Distribución Estadística Global (Univariada)
    - Test de Normalidad (Shapiro-Wilk) en métricas clave
    - Skewness (Asimetría) y Kurtosis

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración Visual Profesional
sns.set_theme(style="whitegrid", context="notebook")
plt.rcParams['figure.figsize'] = (14, 7)
pd.set_option('display.max_columns', None) # Para ver todas las columnas en los describe()

# ==============================================================================
# 0. CARGA DE DATOS (Input del ETL)
# ==============================================================================
INPUT_FILE = '../services/ingestion/data_drop/fbref_big5_players_24_25_clean.csv'

try:
    df = pd.read_csv(INPUT_FILE)
    print(f"✅ BASE CARGADA: {df.shape[0]} registros | {df.shape[1]} variables")
except FileNotFoundError:
    # Fallback local
    df = pd.read_csv('fbref_big5_players_24_25_clean.csv')
    print(f"✅ BASE CARGADA (LOCAL): {df.shape[0]} registros")

df.head(-10)



✅ BASE CARGADA: 3006 registros | 153 variables


,player_name,nation,main_position_group,squad,league_id,age,record_type,matches_played,starts,minutes_played,goals,assists,xg,xa,xa.1,tackles_total,interceptions,progressive_passes,saves,Rk,Pos,born,nineties,goals_assists,G-PK,pk_goals,pk_attempts,yellow_cards,red_cards,npxg,npxG+xAG,progressive_carries,progressive_runs,G+A-PK,xG+xAG,shots_total,shots_on_target,shots_on_target_pct,Sh/90,SoT/90,goals_per_shot,goals_per_sot,avg_shot_distance,free_kick_shots,npxG/Sh,G-xG,np:G-xG,passes_completed,passes_attempted,pass_completion_pct,pass_total_distance,pass_progressive_distance,assists_minus_xa,key_passes,1/3,passes_penalty_area,crosses_penalty_area,Live,Dead,TB,Sw,Crs,TI,CK,In,Out,Str,offsides,blocks,SCA,SCA90,PassLive,PassDead,TO,fouls_drawn,Def,GCA,GCA90,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,Tkl%,Lost,Pass,tackles_interceptions,clearances,errors_leading_to_goal,touches,Def Pen,Att Pen,dribbles_completed,dribble_success_pct,Tkld,Tkld%,carries,CPA,miscontrols,dispossessed,Rec,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,second_yellow_card,fouls_committed,pk_won,pk_conceded,own_goals,ball_recoveries,Won,Won%,goals_against,goals_against_p90,shots_on_target_against,save_pct,wins,draws,losses,clean_sheets,clean_sheet_pct,pk_allowed,pk_saved,pk_missed,psxg,psxg_per_sot,psxg_plus_minus,/90,Att (GK),Thr,launch_pct,avg_pass_length,Opp,crosses_stopped,crosses_stopped_pct,def_actions_outside_box,def_actions_outside_box_p90,avg_keeper_sweeper_dist,team_order,season_id
0,Max Aarons,ENG,DF,Bournemouth,Premier League,24.0,Parcial,3,1,86,0,0,0.0,0.0,0.0,2,1,8,0.0,1,DF,2000.0,1.0,0,0,0,0,0,0,0.0,0.0,1,3,0.00,0.00,0,0,NaN,0.00,0.0,NaN,NaN,NaN,0,NaN,0.0,0.0,50,63,79.4,887,361,0.0,0,8,0,0,51,12,0,0,2,10,0,0,0,0,0,1,2,2.09,2,0,0,0,0,0,0.00,2,1,1,0,100.0,0,2,3,0,0,73,2,0,0,0.0,1,50.0,41,0,1,0,40,29.0,2.5,61.0,0,2,13.0,11,0.67,2,0,2,2.09,1.82,2.3,0.3,2.0,2.12,0,0,0,0,0,7,0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-1
1,Max Aarons,ENG,DF,Valencia,La Liga,24.0,Parcial,4,1,120,0,0,0.0,0.0,0.0,4,0,6,0.0,2,"DF,MF",2000.0,1.3,0,0,0,0,2,0,0.0,0.0,0,10,0.00,0.02,0,0,NaN,0.00,0.0,NaN,NaN,NaN,0,NaN,0.0,0.0,47,66,71.2,705,190,0.0,1,2,0,0,54,12,0,0,5,11,0,0,0,0,0,1,1,0.75,1,0,0,0,0,0,0.00,4,2,2,0,80.0,1,1,4,3,0,85,1,0,1,25.0,2,50.0,46,0,3,1,49,30.0,3.5,73.0,0,3,16.0,14,0.75,1,3,-2,-1.50,-1.28,1.5,3.7,-2.3,-1.69,0,0,0,0,0,7,2,66.7,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,-1
2,Rodrigo Abajas,ESP,DF,Valencia,La Liga,21.0,Parcial,1,1,65,0,0,0.1,0.0,0.0,3,1,2,0.0,3,DF,2003.0,0.7,0,0,0,0,1,0,0.1,0.1,3,3,0.00,0.10,1,0,0.0,1.38,0.0,0.00,NaN,24.5,0,0.07,-0.1,-0.1,17,29,58.6,268,110,0.0,0,0,0,0,21,8,0,0,1,8,0,0,0,0,0,2,0,0.00,0,0,0,0,0,0,0.00,2,2,1,0,100.0,0,1,4,0,0,36,1,1,1,100.0,0,0.0,13,1,0,2,16,65.0,1.9,65.0,0,0,NaN,8,0.00,1,2,-1,-1.38,-1.14,1.4,0.7,0.7,0.93,0,2,0,0,0,2,0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-1
3,James Abankwah,IRL,DF,Udinese,Serie A,20.0,Parcial,6,0,88,0,0,0.1,0.0,0.0,4,1,4,0.0,4,"DF,MF",2004.0,1.0,0,0,0,0,1,0,0.1,0.1,3,1,0.00,0.06,1,0,0.0,1.02,0.0,0.00,NaN,15.0,0,0.06,-0.1,-0.1,36,46,78.3,614,206,0.0,0,2,0,0,45,1,0,0,0,0,0,0,0,0,0,0,1,1.02,1,0,0,0,0,0,0.00,2,4,0,0,66.7,1,1,5,3,0,65,8,2,0,NaN,0,NaN,29,1,1,3,34,15.0,2.6,NaN,0,6,15.0,12,1.67,2,0,2,2.05,2.50,0.6,1.5,-0.9,-0.91,0,4,0,0,0,7,2,50.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-1
4,Keyliane Abdallah,FRA,FW,Marseille,Ligue 1,18.0,Parcial,1,0,3,0,0,0.0,0.0,0.0,1,0,0,0.0,5,FW,2006.0,0.0,0,0,0,0,0,0,0.0,0.0,1,0,0.00,0.00,0,0,NaN,0.00,0.0,NaN,NaN,NaN,0,NaN,0.0,0.0,2,2,100.0,41,0,0.0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0.00,1,1,0,0,100.0,0,0,1,0,0,4,0,0,0,NaN,0,NaN,1,0,1,0,3,3.0,0.1,NaN,0,1,3.0,7,3.00,0,0,0,0.00,-0.79,0.0,0.0,0.0,-0.49,0,0,0,0,0,

In [ ]:
# ==============================================================================
# PARTE 1: ANÁLISIS GENERAL (MACRO-ESTRUCTURA)
# ==============================================================================
print("\n" + "="*50)
print("1. VISIÓN GENERAL (HEALTH CHECK)")
print("="*50)

# A. Integridad de la Base (Nulos)
# Nota: Como hicimos fillna(0) en el ETL, buscamos "ceros excesivos" que podrían ser nulos disfrazados
print("\n--- A. Integridad de Datos ---")
nulos = df.isnull().sum().sum()
print(f"Total de valores Nulos (NaN) en la base: {nulos}")
print("(Si es 0, es porque el ETL hizo su trabajo rellenando con ceros)")

# B. Estadística Descriptiva Global
print("\n--- B. Resumen Estadístico Global (Top Variables) ---")
# Seleccionamos algunas clave para no saturar la pantalla
cols_desc = ['age', 'minutes_played', 'goals', 'assists', 'xg', 'tackles_total', 'saves']
print(df[cols_desc].describe().round(2))

# C. Distribución de Minutos (El Filtro Maestro)
plt.figure(figsize=(12, 5))
sns.histplot(df['minutes_played'], bins=60, kde=True, color='#2c3e50')
plt.axvline(450, color='red', linestyle='--', label='Frontera de Relevancia (450 min)')
plt.title('1.C. Histograma de Minutos: La Montaña de los Suplentes', fontsize=14, fontweight='bold')
plt.legend()
plt.show()




In [ ]:
# ==============================================================================
# PARTE 2: ANÁLISIS INTRA-GRUPAL (POR POSICIONES)
# ==============================================================================
print("\n" + "="*50)
print("2. ANÁLISIS INTRA-GRUPAL (SEGREGACIÓN POSICIONAL)")
print("="*50)

# Filtramos "ruido" para este análisis (Jugadores > 450 min)
df_core = df[df['minutes_played'] > 450].copy()
print(f"Analizando subconjunto de {len(df_core)} jugadores con > 450 minutos.")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# A. Grupo Delanteros (FW) - Eficiencia
sns.boxplot(data=df_core, x='main_position_group', y='goals', ax=axes[0,0], palette='viridis')
axes[0,0].set_title('Distribución de GOLES por Posición')

# B. Grupo Creativos (MF) - Progresión
sns.boxplot(data=df_core, x='main_position_group', y='progressive_passes', ax=axes[0,1], palette='mako')
axes[0,1].set_title('Distribución de PASES PROGRESIVOS por Posición')

# C. Grupo Defensivo (DF) - Destrucción
sns.boxplot(data=df_core, x='main_position_group', y='tackles_total', ax=axes[1,0], palette='rocket')
axes[1,0].set_title('Distribución de TACKLES por Posición')

# D. Grupo Porteros (GK) - Trabajo
# Filtramos solo GK para que el gráfico no quede vacío en otras posiciones
df_gk = df_core[df_core['main_position_group'] == 'GK']
if not df_gk.empty:
    sns.violinplot(data=df_gk, y='saves', ax=axes[1,1], color='cyan')
    axes[1,1].set_title('Densidad de ATAJADAS (Solo Arqueros)')
else:
    axes[1,1].text(0.5, 0.5, 'Sin datos de GK suficientes', ha='center')

plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================================
# PARTE 3: ANÁLISIS INTER-GRUPOS Y CONSISTENCIA (MICRO)
# ==============================================================================
print("\n" + "="*50)
print("3. ANÁLISIS DE CONSISTENCIA Y RELACIONES (CRUCE DE VARIABLES)")
print("="*50)

# A. Correlaciones Tácticas (Heatmap)
# ¿Se mueven juntas las métricas defensivas?
cols_def = ['tackles_total', 'interceptions', 'blocks', 'ball_recoveries', 'yellow_cards']
corr_mat = df_core[cols_def].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_mat, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
plt.title('3.A. Matriz de Coherencia Defensiva', fontsize=14, fontweight='bold')
plt.show()

# B. Scatter de Anomalías: Goles vs xG
# Buscamos "Overperformers" (Talento/Suerte) y "Underperformers" (Mala racha)
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_core, x='xg', y='goals', hue='main_position_group', alpha=0.6, palette='deep')
# Línea de identidad
limit = max(df_core['goals'].max(), df_core['xg'].max())
plt.plot([0, limit], [0, limit], 'r--', alpha=0.5, label='Rendimiento Esperado')
plt.title('3.B. Consistencia Ofensiva: Realidad vs Expectativa', fontsize=14, fontweight='bold')
plt.legend()
plt.show()

# C. Detección de Anomalías Específicas (El caso del "Portero Loco")
# Pregunta del usuario: ¿Hay porteros con muchas tarjetas amarillas?
print("\n--- 3.C. Detección de Anomalías: Porteros Indisciplinados ---")
gks_cards = df_core[df_core['main_position_group'] == 'GK'][['player_name', 'squad', 'yellow_cards']].sort_values('yellow_cards', ascending=False)
top_bad_boys = gks_cards.head(5)

if not top_bad_boys.empty:
    print("Top 5 Arqueros con más amarillas:")
    print(top_bad_boys)
    
    # Gráfico rápido si hay datos
    plt.figure(figsize=(8, 4))
    sns.barplot(data=top_bad_boys, x='yellow_cards', y='player_name', palette='Reds_r')
    plt.title('Anomalía: Arqueros con más Tarjetas Amarillas')
    plt.show()
else:
    print("No se encontraron arqueros con tarjetas en el corte seleccionado.")

print("\n" + "="*50)
print("✅ EDA FINALIZADO. DIAGNÓSTICO COMPLETO.")
print("="*50)